# Tutorial on using multiple models for evaluation
- This tutorial uses the file `multiple_model_utils.py` (in the same directory as this one) which contains the functions for training and evaluating multiple models. Please refer to the file for the implementation details!

In [1]:
from __future__ import annotations
import os
import sys

from pathlib import Path
import json

import glob
# change the path to the backend directory
sys.path.append(os.path.join(os.path.dirname("."), '../../backend/'))
from modules.utils import load_config_and_device
from multiple_model_utils import *


/Users/smukherjee/.pyenv/versions/3.10.14/envs/openml/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
new_path = Path("../../backend/")

config = load_config_and_device(str(new_path / "config.json"), training = True)

config["type_of_data"] = "dataset"
config["training"] = True

[INFO] Finding device.
[INFO] Device found: mps


## Defining the models used
- Embedding models are any from Huggingface hub
- LLM models are any from Ollama library

In [9]:
list_of_embedding_models = ["BAAI/bge-large-en-v1.5", "BAAI/bge-small-en-v1.5", "BAAI/bge-base-en-v1.5"]
# list_of_llm_models = ["qwen2:1.5b", "llama3"]
# list_of_llm_models = ["llama3"]
list_of_llm_models = ["qwen2:1.5b"]

## Downloading the LLM models
- PLEASE MAKE SURE YOU HAVE DOWNLOADED OLLAMA
- Linux/Unix : ```curl -fsSL https://ollama.com/install.sh | sh```

In [4]:
ollama_setup(list_of_llm_models)

Waiting for Ollama server to be active...
NAME         	ID          	SIZE  	MODIFIED      


Error: listen tcp 127.0.0.1:11434: bind: address already in use
pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest ⠇ pulling manifest ⠏ pulling manifest ⠋ pulling manifest ⠙ pulling manifest ⠹ pulling manifest ⠸ pulling manifest ⠼ pulling manifest ⠴ pulling manifest ⠦ pulling manifest ⠧ pulling manifest 
pulling 405b56374e02... 100% ▕████████████████▏ 934 MB                         
pulling 62fbfd9ed093... 100% ▕████████████████▏  182 B                         
pulling c156170b718e... 100% ▕██████

## Setup evaluation data
### If you used tools/app.py to generate evaluation data
- You can ignore this and use the data generated by the tool
### If you did not
- You can use evaluation data of the format {"id": ["tag1", "tag2"] } and save it as a json file
- eg: ```{"43843": ["Climate change"], "43365": ["COVID-19"], "43684": ["COVID-19"]}```

In [5]:
query_templates = [
    "Find me a dataset about",
    "Dataset on",
    "list datasets about",
    "Can you locate a dataset on",
    "I'm looking for a dataset related to",
    "Please help me find a dataset concerning",
    "Is there a dataset available for",
    "Could you provide a dataset on",
    "I need a dataset regarding",
    "Can you source a dataset about",
    "I'd like to get a dataset about",
    "Can you identify a dataset related to",
    "Do you have access to a dataset on"
]

In [6]:
with open("../../data/evaluation/merged_labels.json", "r") as f:
    merged_labels = json.load(f)
    # get the dataset ids we want out evaluation to be based on (these are dataset ids for the openml datasets)
    subset_ids = list(merged_labels.keys())

In [7]:
df_queries = get_dataset_queries(subset_ids, query_templates, merged_labels)
queries = df_queries["query"].values

## Test accuracy with different embeddings

In [10]:
run_experiments(config, new_path,queries, 
list_of_embedding_models,list_of_llm_models, subset_ids, use_cached_experiment= True)

LLM Models: 100%|██████████| 1/1 [00:00<00:00, 1911.72it/s]


Experiment BAAI_bge-large-en-v1.5_qwen2_1.5b already exists. Skipping... To disable this behavior, set use_cached_experiment = False


LLM Models: 100%|██████████| 1/1 [00:00<00:00, 7037.42it/s]


Experiment BAAI_bge-small-en-v1.5_qwen2_1.5b already exists. Skipping... To disable this behavior, set use_cached_experiment = False


[INFO] Training is set to True.
[INFO] Initializing cache.
[INFO] Getting dataset metadata from OpenML.



QUEUEING TASKS | : 100%|██████████| 5703/5703 [00:00<00:00, 215078.01it/s]





































PROCESSING TASKS | : 100%|██████████| 5703/5703 [00:10<00:00, 525.29it/s] 

COLLECTING RESULTS | : 100%|██████████| 5703/5703 [00:00<00:00, 559574.14it/s]


[INFO] Saving metadata to file.
[INFO] Loading model...
[INFO] Model loaded.
[INFO] Generating unique documents. Total documents: 960
Number of unique documents: 933 vs Total documents: 960





































Batches: 100%|██████████| 16/16 [00:21<00:00,  1.32s/it]































Batches: 100%|██████████| 14/14 [00:14<00:00,  1.01s/it]

100%|██████████| 2/2 [00:35<00:00, 17.92s/it]

Batches: 100%|██████████| 1/1 [00:00<00:00, 10.05it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 64.70it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 15.19it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 15.45it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 47.69it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 15.40it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 68.05it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 63.03it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 52.88it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 46.99it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 68.09it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 16.03it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 63.69it/s]

Batches: 100%|██████████| 1/1 [00:00<00:00, 

## Load the results and evaluate

In [16]:
# glob all csv files in the experiments directory
experiment_dir = Path(f"../../data/experiments/")
csv_files = glob.glob(str(experiment_dir / "*/results.csv"))

In [17]:
results_dict = create_results_dict(csv_files, df_queries)

In [18]:
pd.DataFrame.from_dict(results_dict, orient='index')

,correct,wrong
BAAI_bge-base-en-v1.5_qwen2_1.5b,550,100
BAAI_bge-large-en-v1.5_qwen2_1.5b,579,71


In [ ]:
pd.DataFrame.from_dict(results_dict, orient='index').to_csv("../../data/experiments/results.csv")